In [1]:
import os
import datetime
from datetime import timedelta
import quandl
import matplotlib.pyplot as plt

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy as sp
import plotnine as p9
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import zscore

import warnings
import functools
import wrds
# Ignore all warnings
warnings.filterwarnings("ignore")

In [2]:
df_prices = pd.read_parquet('data/stock_price.parquet')
df_prices

,date,permno,prc,bid,ask,ticker,comnam
0,2017-12-01,10138,102.79,102.76,102.77,TROW,T ROWE PRICE GROUP INC
1,2017-12-01,10065,14.85,14.84,14.85,ADX,ADAMS EXPRESS CO
2,2017-12-04,10065,14.84,14.83,14.84,ADX,ADAMS EXPRESS CO
3,2017-12-05,10065,14.82,14.81,14.82,ADX,ADAMS EXPRESS CO
4,2017-12-06,10065,14.79,14.79,14.80,ADX,ADAMS EXPRESS CO
...,...,...,...,...,...,...,...
6513913,2023-02-22,71079,14.34,14.35,14.37,RVT,ROYCE VALUE TRUST INC
6513914,2023-02-23,71079,14.41,14.39,14.41,RVT,ROYCE VALUE TRUST INC
6513915,2023-02-24,71079,14.28,14.28,14.30,RVT,ROYCE VALUE TRUST INC
6513916,2023-02-27,71079,14.34,14.34,14.35,RVT,ROYCE VALUE TRUST INC


In [3]:
df_ED = pd.read_parquet('tickers_ED_20182023.parquet')
df_ED

,Short_ticker,Name,Ticker,Date,Event Type
0,A,Agilent Technologies Inc,A US,2018-02-14 16:05:00,ER
1,A,Agilent Technologies Inc,A US,2018-05-14 16:05:00,ER
2,A,Agilent Technologies Inc,A US,2018-08-14 16:05:00,ER
3,A,Agilent Technologies Inc,A US,2018-11-19 16:05:00,ER
4,A,Agilent Technologies Inc,A US,2019-02-20 16:05:00,ER
...,...,...,...,...,...
19471,ZTS,Zoetis Inc,ZTS US,2023-02-14 07:00:00,ER
19472,ZTS,Zoetis Inc,ZTS US,2023-05-04 07:00:00,ER
19473,ZTS,Zoetis Inc,ZTS US,2023-08-08 07:00:00,ER
19474,ZTS,Zoetis Inc,ZTS US,2023-11-02 07:00:00,ER


In [4]:
# filter stock prices to only include those in the ED list
unique_tickers = df_ED['Short_ticker'].unique()
df_prices = df_prices[df_prices['ticker'].isin(unique_tickers)]
df_prices.sort_values(by=['ticker', 'date'], inplace=True)
df_prices.reset_index(drop=True, inplace=True)
df_prices

,date,permno,prc,bid,ask,ticker,comnam
0,2017-12-01,87432,68.69000,68.67000,68.68,A,AGILENT TECHNOLOGIES INC
1,2017-12-04,87432,66.24000,66.23000,66.24,A,AGILENT TECHNOLOGIES INC
2,2017-12-05,87432,65.83000,65.80000,65.81,A,AGILENT TECHNOLOGIES INC
3,2017-12-06,87432,65.89000,65.89000,65.90,A,AGILENT TECHNOLOGIES INC
4,2017-12-07,87432,66.71000,66.71000,66.72,A,AGILENT TECHNOLOGIES INC
...,...,...,...,...,...,...,...
965742,2023-02-22,13788,166.42999,166.49001,166.50,ZTS,ZOETIS INC
965743,2023-02-23,13788,169.50000,169.47000,169.48,ZTS,ZOETIS INC
965744,2023-02-24,13788,166.33000,166.28000,166.33,ZTS,ZOETIS INC
965745,2023-02-27,13788,165.47000,165.50999,165.52,ZTS,ZOETIS INC


In [5]:
# include stock prices for 5 days before or after the ED date

df_ED['Date'] = pd.to_datetime(df_ED['Date'])
df_ED['Date'] = df_ED['Date'].dt.date
df_ED['Date'] = df_ED['Date'].astype(str)
df_prices['date'] = df_prices['date'].astype(str)
# df_prices['date'] = pd.to_datetime(df_prices['date'])


In [6]:

result = pd.DataFrame(columns=['date', 'permno','prc', 'bid', 'ask', 'ticker', 'comnam'])

for ticker in df_ED['Short_ticker'].unique():
    date_list = df_ED[df_ED['Short_ticker'] == ticker]['Date'].tolist()

    ticker_prices = df_prices[df_prices['ticker'] == ticker]
    matched_rows = ticker_prices[ticker_prices['date'].isin(date_list)]

    # prices on ED date have to be greater than 20
    matched_rows = matched_rows[matched_rows['prc'] >= 20]

    indices = matched_rows.index.tolist()
    
    for index in indices:
        start_index = max(0, index - 5)
        end_index = min(index + 5, ticker_prices.index.max())
        selected_rows = df_prices.iloc[start_index:end_index+1]

        # Concatenate the selected_rows dataframe with the result_df dataframe
        result = pd.concat([result, selected_rows])

In [7]:
result

,date,permno,prc,bid,ask,ticker,comnam
45,2018-02-07,87432,68.06000,68.06000,68.07000,A,AGILENT TECHNOLOGIES INC
46,2018-02-08,87432,65.05000,65.03000,65.05000,A,AGILENT TECHNOLOGIES INC
47,2018-02-09,87432,66.98000,66.97000,66.98000,A,AGILENT TECHNOLOGIES INC
48,2018-02-12,87432,68.43000,68.40000,68.41000,A,AGILENT TECHNOLOGIES INC
49,2018-02-13,87432,68.34000,68.33000,68.34000,A,AGILENT TECHNOLOGIES INC
...,...,...,...,...,...,...,...
965738,2023-02-15,13788,175.02000,175.10001,175.11000,ZTS,ZOETIS INC
965739,2023-02-16,13788,173.92999,173.97000,174.02000,ZTS,ZOETIS INC
965740,2023-02-17,13788,172.03000,172.09000,172.10001,ZTS,ZOETIS INC
965741,2023-02-21,13788,169.84000,169.87000,169.89999,ZTS,ZOETIS INC


In [8]:
result['date'] = pd.to_datetime(result['date'])
result[result['ticker'] == 'AAPL']

,date,permno,prc,bid,ask,ticker,comnam
5308,2018-01-25,14593,171.11000,171.10001,171.11000,AAPL,APPLE INC
5309,2018-01-26,14593,171.50999,171.39000,171.39999,AAPL,APPLE INC
5310,2018-01-29,14593,167.96001,167.95000,167.96001,AAPL,APPLE INC
5311,2018-01-30,14593,166.97000,166.96001,166.97000,AAPL,APPLE INC
5312,2018-01-31,14593,167.42999,167.45000,167.47000,AAPL,APPLE INC
...,...,...,...,...,...,...,...
6573,2023-02-03,14593,154.50000,154.49001,154.50000,AAPL,APPLE INC
6574,2023-02-06,14593,151.73000,151.67999,151.69000,AAPL,APPLE INC
6575,2023-02-07,14593,154.64999,154.63000,154.66000,AAPL,APPLE INC
6576,2023-02-08,14593,151.92000,151.94000,151.96001,AAPL,APPLE INC


In [11]:
result.to_parquet('stock_price_ED_20182023.parquet')